# Домашнее задание к лекции "Основы веб-скрапинга и работы с API"

## Задание 1. 

### Обязательная часть

Будем парсить страницу со свежеми новостям на [habr.com/ru/all/](https://habr.com/ru/all/).

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

`KEYWORDS = ['python', 'парсинг']`

 Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы). 
 
В итоге должен формироваться датафрейм со столбцами: <дата> - <заголовок> - <ссылка>.

### Дополнительная часть (необязательная)

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст статьи>

In [1]:
import requests
import time
import pandas as pd
from datetime import datetime, timedelta
from bs4 import BeautifulSoup

In [2]:
KEYWORDS = ['python', 'data mining', 'html']

In [3]:
req = requests.get('https://habr.com/ru/all/')
soup = BeautifulSoup(req.text, 'html.parser')

In [4]:
posts = soup.find_all('article', class_='post')
# posts

In [5]:
for post in posts:
    post_id = post.parent.attrs.get('id')
    if not post_id:
        continue
    post_id = int(post_id.split('_')[-1])
#     print('post', post_id)
    hubs = post.find_all('a', class_='hub-link')

In [6]:
# добавляем извлечение хабов из постов, чтобы отбирать только нужные
list_ = []

for post in posts:
    post_id = post.parent.attrs.get('id')
    # если идентификатор не найден, это что-то странное, пропускаем
    if not post_id:
        continue
    post_id = int(post_id.split('_')[-1])
    hubs = post.find_all('a', class_='hub-link')
    
    for hub in hubs:
            hub_lower = hub.text.lower()
            # ищем вхождение хотя бы одного желаемого хаба
            if any([hub_lower in words for words in KEYWORDS]):
               # пост нам интересен - делаем с ним все что захотим:
               # можно отправит в телеграм уведомление, можно на почту и т.п.
                title_element = post.find('a', class_='post__title_link')
                a = title_element.attrs.get('href')
                soup = BeautifulSoup(requests.get(a).text, 'html.parser')
                time.sleep(0.3)
                text = soup.find('div', class_='post__text post__text-html post__text_v1').text
                title_element_2 = post.find('a', class_='post__title_link').text
                data = post.find('span', class_='post__time').text
                row = {'data': data, 'title': title_element_2, 'href': a, 'text': text} 
                list_.append(row)        
                # так как пост уже нам подошел - дальше нет смысла проверять хабы
                break         

Вариант с плохим форматом даты размещения.

In [7]:
df = pd.DataFrame(list_)
df

,data,title,href,text
0,вчера в 22:51,Тесты на pytest с генерацией отчетов в Allure ...,https://habr.com/ru/post/513432/,Этот текст предназначен для начинающих тестиро...


In [8]:
def data(string):
    if 'сегодня' in string:
        date = datetime.strptime(str(datetime.today()).split(' ')[0], '%Y-%m-%d')
        return date
    elif 'вчера' in string: 
        date_2 = datetime.strptime(str(datetime.today()).split(' ')[0], '%Y-%m-%d') - timedelta(days=1)
        return date_2
    else:
        x = string.split(' ') 
        date_3 = []
        [date_3.append(x[3-i]) for i, y in enumerate(x)]
        return ' '.join(date_3[1:4])
        # В блоке else нужно подумать как переводить месяц прописанный русскими буквами в числовое значение 
        # чтобы потом возвращать дату ка в блоке if и elif.
        # Как вариант можно через условные конструкции.

In [9]:
df['date'] = df['data'].apply(data)
df = df.drop('data', 1)

Вариант с хорошим форматом даты размещения.

In [10]:
neworder = ['date','title','href', 'text']
df = df.reindex(columns=neworder)
df

,date,title,href,text
0,2020-08-01,Тесты на pytest с генерацией отчетов в Allure ...,https://habr.com/ru/post/513432/,Этот текст предназначен для начинающих тестиро...


## Задание 2.

### Обязательная часть

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса [Avast Hack Ckeck](https://www.avast.com/hackcheck/).
Список email-ов задаем переменной в начале кода:  
`EMAIL = [xxx@x.ru, yyy@y.com]`

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>.

In [11]:
email_list = ['eterin@agriculture.ru',
         'lawpro-707@list.ru',
         'solowey011@yandex.ru',
         'ahds@aol.com',
         'akas@aol.com',
         'workpost@fxt.ru'
]

In [12]:
def check_email_list(email_list):
    list_of_leaks = []

    for email in email_list:
        url = 'https://digibody.avast.com/v1/web/leaks'
        params = {'email': email}
        req = requests.post(url, json=params)
        time.sleep(0.3)
        leaks = req.json()['value']

        for leak in leaks:
            email_2 = leak['username']
            date = leak['leak_info']['date']
            source = leak['leak_info']['title']
            description = leak['leak_info']['description'] 
            row = {'email': email_2, 'date': date, 'source': source, 'description': description} 
            list_of_leaks.append(row)
    df = pd.DataFrame(list_of_leaks)   
    return df      

In [13]:
check_email_list(email_list)

,email,date,source,description
0,dark_salamon,1565222400000,Borealis.su,"At an unconfirmed date, Russian Minecraft serv..."
1,lawpro-707@list.ru,1588204800000,Payad,"At an unconfirmed date, the advertising revenu..."
2,solowey011@yandex.ru,1549411200000,Collection #5 Combo List,"On January 7, 2019, an online user named Sanix..."
3,solowey011@yandex.ru,1548720000000,Collection #2 Combo List,"On January 7, 2019, an online user named Sanix..."
4,solowey011@yandex.ru,1549411200000,2019 Antipublic Combo List,"On January 7, 2019, an online user named Sanix..."
5,solowey011@yandex.ru,1549411200000,Collection #4 Combo List,"On January 7, 2019, an online user named Sanix..."
6,ahds@aol.com,1513900800000,Combolist of 1.4 Billion Credentials,The proliferation of stolen or leaked database...
7,ahds@aol.com,1549411200000,Collection #4 Combo List,"On January 7, 2019, an online user named Sanix..."
8,ahds@aol.com,1493856000000,Active Account Cracking List,The proliferation of stolen or leaked-breach d...
9,ahds@aol.com,1516406400000,Army Force Warland,"In 2017, Army Force Warland's database was all..."


### Дополнительная часть (необязательная)

Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.  
Документация к API VK: https://vk.com/dev/methods
, вам поможет метод [wall.get](https://vk.com/dev/wall.get)```
GROUP = 'netology'
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ
```

В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>.

In [19]:
URL = 'https://api.vk.com/method/newsfeed.search?'
GROUP = 'netology'
TOKEN = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'
VERSION = '5.103'
SLEEP = 0.33

In [20]:
params = {
    'access_token': TOKEN,
    'v': VERSION,
    'q': GROUP,
    'count': 50
}

In [21]:
res = requests.get(URL, params)
# res.text
# res.json()

In [22]:
# res.json()['response']['items']

In [23]:
df_3 = pd.DataFrame(res.json()['response']['items'])
df_3.head()

,id,date,owner_id,from_id,post_type,text,marked_as_ads,attachments,comments,likes,reposts,views,is_favorite,signer_id
0,243,1596294720,-186521205,-186521205,post,Как начинающему специалисту получить первую ра...,0.0,"[{'type': 'photo', 'photo': {'album_id': -7, '...",{'count': 0},{'count': 0},{'count': 0},{'count': 2},False,NaN
1,85,1596283206,-194003840,-194003840,post,"Привет, ребят)\nУже наступил август и это отли...",0.0,"[{'type': 'link', 'link': {'url': 'https://net...",{'count': 0},{'count': 2},{'count': 0},{'count': 20},False,NaN
2,9665,1596276216,-60643309,-60643309,post,"Чтобы писать интересно и легко, начинающему ко...",0.0,"[{'type': 'link', 'link': {'url': 'https://m.v...",{'count': 0},{'count': 38},{'count': 11},{'count': 1103},False,NaN
3,336,1596256949,-154888653,-154888653,post,,0.0,"[{'type': 'link', 'link': {'url': 'https://m.v...",{'count': 0},{'count': 0},{'count': 0},{'count': 28},False,223781308.0
4,433,1596252494,-123717472,-123717472,post,"Давно мы не рассказывали о том, как проходит р...",0.0,"[{'type': 'link', 'link': {'url': 'https://m.v...",{'count': 0},{'count': 0},{'count': 0},{'count': 1},False,NaN


In [24]:
news_netology = df_3[['date', 'text']]
news_netology

,date,text
0,1596294720,Как начинающему специалисту получить первую ра...
1,1596283206,"Привет, ребят)\nУже наступил август и это отли..."
2,1596276216,"Чтобы писать интересно и легко, начинающему ко..."
3,1596256949,
4,1596252494,"Давно мы не рассказывали о том, как проходит р..."
5,1596252375,"Давно мы не рассказывали о том, как проходит р..."
6,1596252345,"Давно мы не рассказывали о том, как проходит р..."
7,1596252313,"Давно мы не рассказывали о том, как проходит р..."
8,1596211204,«SMM полочкам для арт-бизнеса»! Авторский курс...
9,1596197527,https://netology.ru/free-lessons/stante-litsom...
